## Explanation
A lot of functions are defined. They can be split up in several parts:
1. Loading of the data. Loads the right data and the parameters etc
2. Training of the model. Trains the selected model.
3. Calculation of the CI and the pvalue of AUC: As it states
4. Plotting algorithms: Plot the ROC curves, PR curves, Feature importance bars and number needed to screen

These are called in the final section and gives the results.

The algorithms are tested on a server online, indicating the need for installing several modules, as well as refering to ./datafolder, where the files were stored. For replication, this has to be altered.

# Install needed modules

%pip install --upgrade pip
%pip install --upgrade Pillow

%pip install import-ipynb
%pip install sklearn
%pip install xgboost
%pip install mlxtend
#!pip install pickle

# Define functions

### Loading of the data, FFS and hyperparameter info

In [ ]:
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd

def loadmodules(setNumber, modelName):
    
    # First, a test split is made (the same as during training). On this test data, no training has been performed.
    # After, extra data is loaded, from which extra controls are added to the test data to increase the ratio cases:controls 
    
    with open('./datafolder/DataMatrix{}.pickle'.format(setNumber), 'rb') as f:
        X, y = pickle.load(f)
        X.replace(True,1, inplace = True)
        X.replace(False,0, inplace = True)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify = y)
    del X, y

    with open('./datafolder/DataMatrix{}_extra.pickle'.format(setNumber), 'rb') as f:
        X, y = pickle.load(f)
        X.replace(True,1, inplace = True)
        X.replace(False,0, inplace = True)
    
    from random import sample, seed
    
    # Mogelijk is er een andere ratio (bijv 1:5) gebruikt bij training
    # Hier worden die controles verwijderd uit de mogelijke lijst met controles (voor de 20% test set)
    # Als useNewRatio = True, trainen we hier ook met een 1:5 ratio. Doen we hier niet
    useNewRatio = False
    new_ratio=5
    len_cases = y_train['target'].sum()
    seed(0)
    toremove_list = sample(list(X.index),len_cases*(new_ratio-1))
    
    if useNewRatio==True:
        X_train = pd.concat([X_train, X.loc[toremove_list]])
        y_train = pd.concat([y_train, y.loc[toremove_list]])
        
    X.drop(index = toremove_list, inplace = True)
    y.drop(index = toremove_list, inplace = True)
    
    # Selecteer at random precies 44 controles (= 1:45 ratio) voor de test set
    len_cases = y_test['target'].sum()
    seed(0)
    tosample_list = sample(list(X.index),len_cases*44)
    X = X.loc[tosample_list]
    y = y.loc[tosample_list]
    
    # Alle extras zijn de 44: Voor trainen wordt dit niet gebruikt, bij testen worden deze bijgevoegd
    X_train_extra = X.copy()
    X_test_extra = X.copy()
    y_train_extra = y.copy()
    y_test_extra = y.copy()
    
    del X, y
    
    with open('./datafolder/FFS_set{}_full_{}'.format(setNumber,modelName),'rb') as handle:
        pipe = pickle.load(handle)
    
    # Find the best parameters as selected by FFS
    full_dict = pipe['sfs'].get_metric_dict()
    max_ind = max(full_dict.keys(), key=(lambda key: full_dict[key]['avg_score']))
    best_features = full_dict[max_ind]['feature_names']
    best_features_names = best_features
    best_features = [int(x) for x in best_features]
    
    # Make datasets with only the best parameters
    X_train = X_train.iloc[:,best_features]
    y_train = y_train.iloc[:,0]
    X_test = X_test.iloc[:,best_features]
    y_test = y_test.iloc[:,0]
    
    X_train_extra = X_train_extra.iloc[:,best_features]
    y_train_extra = y_train_extra.iloc[:,0]
    X_test_extra = X_test_extra.iloc[:,best_features]
    y_test_extra = y_test_extra.iloc[:,0]
    
    with open('./datafolder/Tuning_set{}_{}'.format(setNumber, modelName),'rb') as handle:
        gridsearch = pickle.load(handle)
        
    best_params = {x.replace("classifier__", ""): v for x, v in gridsearch.best_params_.items()}
        
    return X_train, y_train, X_test, y_test, best_params, pipe, X_train_extra, X_test_extra, y_train_extra, y_test_extra


### Calculation of the CI and pvalue of the AUC curve

In [ ]:
def calcAUC_CI(AUC, N1, N2):
    # Calculates the confidence interval of a ROC curve
    import math
    Q1 = AUC / (2-AUC)
    Q2 = (2*math.pow(AUC,2))/(1+AUC)
    
    part1 = AUC * (1-AUC)
    part2 = (N1-1) * (Q1-math.pow(AUC,2))
    part3 = (N2-1) * (Q2-math.pow(AUC,2))
    part4 = N1*N2
    
    SE_AUC = math.sqrt((part1+part2+part3)/part4) 
    
    ci = [AUC - 1.96*SE_AUC, AUC + 1.96*SE_AUC]
    
    return ci

def calc_pvalue_AUCs(ground_truth, predictions_one, predictions_two):
    """
    The implementation of DeLong's method is used as from:
    https://github.com/yandexdataschool/roc_comparison/tree/44fcd23c998b39f30440d833a0820f1c7e6d8bc7
    as on 18 nov 2021.
    """
    
    import pandas as pd
    import numpy as np
    import scipy.stats

    def compute_midrank(x):
        """Computes midranks.
        Args:
           x - a 1D numpy array
        Returns:
           array of midranks
        """
        J = np.argsort(x)
        Z = x[J]
        N = len(x)
        T = np.zeros(N, dtype=np.float)
        i = 0
        while i < N:
            j = i
            while j < N and Z[j] == Z[i]:
                j += 1
            T[i:j] = 0.5*(i + j - 1)
            i = j
        T2 = np.empty(N, dtype=np.float)
        # Note(kazeevn) +1 is due to Python using 0-based indexing
        # instead of 1-based in the AUC formula in the paper
        T2[J] = T + 1
        return T2


    def fastDeLong(predictions_sorted_transposed, label_1_count):
        """
        The fast version of DeLong's method for computing the covariance of
        unadjusted AUC.
        Args:
           predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
              sorted such as the examples with label "1" are first
        Returns:
           (AUC value, DeLong covariance)
        Reference:
         @article{sun2014fast,
           title={Fast Implementation of DeLong's Algorithm for
                  Comparing the Areas Under Correlated Receiver Operating Characteristic Curves},
           author={Xu Sun and Weichao Xu},
           journal={IEEE Signal Processing Letters},
           volume={21},
           number={11},
           pages={1389--1393},
           year={2014},
           publisher={IEEE}
         }
        """
        # Short variables are named as they are in the paper
        m = label_1_count
        n = predictions_sorted_transposed.shape[1] - m
        positive_examples = predictions_sorted_transposed[:, :m]
        negative_examples = predictions_sorted_transposed[:, m:]
        k = predictions_sorted_transposed.shape[0]

        tx = np.empty([k, m], dtype=np.float)
        ty = np.empty([k, n], dtype=np.float)
        tz = np.empty([k, m + n], dtype=np.float)
        for r in range(k):
            tx[r, :] = compute_midrank(positive_examples[r, :])
            ty[r, :] = compute_midrank(negative_examples[r, :])
            tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
        aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
        v01 = (tz[:, :m] - tx[:, :]) / n
        v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
        sx = np.cov(v01)
        sy = np.cov(v10)
        delongcov = sx / m + sy / n
        return aucs, delongcov


    def calc_pvalue(aucs, sigma):
        """Computes log(10) of p-values.
        Args:
           aucs: 1D array of AUCs
           sigma: AUC DeLong covariances
        Returns:
           log10(pvalue)
        """
        l = np.array([[1, -1]])
        z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
        return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


    def compute_ground_truth_statistics(ground_truth):
        assert np.array_equal(np.unique(ground_truth), [0, 1])
        order = (-ground_truth).argsort()
        label_1_count = int(ground_truth.sum())
        return order, label_1_count


    def delong_roc_variance(ground_truth, predictions):
        """
        Computes ROC AUC variance for a single set of predictions
        Args:
           ground_truth: np.array of 0 and 1
           predictions: np.array of floats of the probability of being class 1
        """
        order, label_1_count = compute_ground_truth_statistics(ground_truth)
        predictions_sorted_transposed = predictions[np.newaxis, order]
        aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
        assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
        return aucs[0], delongcov


    def delong_roc_test(ground_truth, predictions_one, predictions_two):
        """
        Computes log(p-value) for hypothesis that two ROC AUCs are different
        Args:
           ground_truth: np.array of 0 and 1
           predictions_one: predictions of the first model,
              np.array of floats of the probability of being class 1
           predictions_two: predictions of the second model,
              np.array of floats of the probability of being class 1
        """
        order, label_1_count = compute_ground_truth_statistics(ground_truth)
        predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
        aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
        return calc_pvalue(aucs, delongcov)
    
    # Actual call to definitions
    pvalue = 10**delong_roc_test(ground_truth, predictions_one, predictions_two)
    return pvalue


### Training of the model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

def trainModel(X_train, y_train, X_test, y_test, X_train_extra, X_test_extra, y_train_extra, y_test_extra, best_params, modelName, ratio = '1to1', teston = '1to1', withcv = False, sortCurve = 'AUC'):

    # Make the model
    categorical = list(X_train.select_dtypes('category').columns)
    numerical = list(X_train.select_dtypes('number').columns)
    transformer = ColumnTransformer(transformers=[('cat', StandardScaler(), categorical),
                                                  ('num', StandardScaler(), numerical)])

    if modelName == 'LR':
        classifier = LogisticRegression(**best_params, random_state = 42)
    elif modelName == 'RF':
        classifier = RandomForestClassifier(**best_params, random_state = 42)
    elif modelName == 'XGB':
        classifier = xgb.XGBClassifier(**best_params, random_state = 42)
    
    # A model is always trained first on non-upsampled training data
    clf = Pipeline([('preprocessing', transformer), ('classifier',classifier)])
    clf.fit(X_train,y_train)
    
    # Afterwards, data is upsampled, to predict on (upsampled) training data (we dont use this) and upsampled test data (all data from this is unseen)
    if ratio == 'upsampled':
        X_train = pd.concat([X_train, X_train_extra])
        X_test = pd.concat([X_test, X_test_extra])
        y_train = pd.concat([y_train, y_train_extra])
        y_test = pd.concat([y_test, y_test_extra])
    
    # Predict the outcome
    y_pred_train = clf.predict_proba(X_train)
    y_pred_test = clf.predict_proba(X_test)
    
    # Create an ROC or an precision recall curve
    if sortCurve == 'AUC':
        fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_pred_train[:, 1])
        fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_pred_test[:, 1])
        fpr_train[-1] = 1.0
        fpr_test[-1] = 1.0
        
    elif sortCurve == 'PR':
        tpr_train, fpr_train, thresholds_train = precision_recall_curve(y_train, y_pred_train[:, 1])
        tpr_test, fpr_test, thresholds_test = precision_recall_curve(y_test, y_pred_test[:, 1])
        fpr_train[0] = 1.0
        fpr_test[0] = 1.0
    
    tpr_train[-1]=1.0
    tpr_test[-1]=1.0
    
    # Get the AUC score
    auc_train = auc(fpr_train, tpr_train)
    auc_test = auc(fpr_test, tpr_test)

    return [fpr_train, tpr_train, thresholds_train, auc_train, y_pred_train[:,1], y_train], [fpr_test, tpr_test, thresholds_test, auc_test, y_pred_test[:,1], y_test], clf
    

### Plot the AUC or PR curve

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
%matplotlib inline
#matplotlib notebook

def plotCurve(plotInfo, curve = 'test', sortCurve = 'AUC'):
    
    # Plot the ROC or PR curve
    # All data is stored in the plotInfo, so it is just plotting

    plots = []
    auc = {}
    curvedict = {'test':1, 'train':0}

    for i in plotInfo.keys():

        plots.append(plt.plot(plotInfo[i][curvedict[curve]][0], plotInfo[i][curvedict[curve]][1], 
                lw = 2, label = 'Data({})set {}, AUC = {}'.format(curve, i, str(plotInfo[i][curvedict[curve]][3])[0:6])))
        
        ind = np.argmax(plotInfo[i][curvedict[curve]][1]-plotInfo[i][curvedict[curve]][0])
        
        if sortCurve =='AUC':
            thres = plotInfo[i][curvedict[curve]][2][ind] 
            y_pred_bin = [1 if x>=thres else 0 for x in plotInfo[i][curvedict[curve]][4]]
            cf_mat = confusion_matrix(plotInfo[i][curvedict[curve]][5], y_pred_bin)
            print('{}'.format(i))
            print(cf_mat)
            print('Accuracy: {}'.format((cf_mat[0,0]+cf_mat[1,1])/np.sum(cf_mat)))
            print('Sensitivity: {}'.format(plotInfo[i][curvedict[curve]][1][ind]))
            print('Specificity: {}'.format(1-plotInfo[i][curvedict[curve]][0][ind]))
            
            n1 = plotInfo[i][curvedict[curve]][5].sum()
            n2 = len(plotInfo[i][curvedict[curve]][5]) - n1
            CI = calcAUC_CI(plotInfo[i][curvedict[curve]][3], n1, n2)
            print('Confidence interval: {}', CI)
    
    if sortCurve =='AUC':
        plt.xlabel('1-Specificity')
        plt.ylabel('Sensitivity')
        plots.append(plt.plot([0,1],[0,1], lw = 2, label = 'Random'))
        
    elif sortCurve == 'PR':
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plots.append(plt.plot([0,1],[plotInfo[i][1][5].sum()/plotInfo[i][1][5].size,plotInfo[i][1][5].sum()/plotInfo[i][1][5].size], lw = 2, label = 'Random'))
    
    legax = []
    for numax in plots:
        legax.append(numax[0])

    plt.legend(handles = legax)
    
    plt.grid(color='gray', linestyle='-', linewidth=0.3)
    plt.title('1:45 ratio, -24 to -12 months')

### Plot the number of patients called up vs number of patients identified

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import math
%matplotlib inline

def plotNumtoscreenCurve(plotInfo, curve = 'test'):
    
    # Plot the number needed to screen based on the ROC curve.
    # From the ROC curve, sensitivity and specificity at each point is given.
    # Based on sens and spec, the total num of patients correctly screened vs. faulty positives can be derived

    plots = []
    curvedict = {'test':1, 'train':0}

    for i in plotInfo.keys():

        toScreen = [[],[]]
        for j in plotInfo[i][curvedict[curve]][2]:
            y_pred_bin = [1 if x>=j else 0 for x in plotInfo[i][curvedict[curve]][4]]
            cf_mat = confusion_matrix(plotInfo[i][curvedict[curve]][5], y_pred_bin)
            toScreen[0].append(cf_mat[1][1])
            toScreen[1].append(cf_mat[0][1] + cf_mat[1][1])
            
        plots.append(plt.plot(toScreen[1], toScreen[0], 
                lw = 2, label = 'Data({})set {}'.format(curve, i)))
        
        aantal_pat = toScreen[0][-1]
        
        ten_p = math.floor(aantal_pat/10)
        for ind,values in enumerate(toScreen[0]):
            if values >= ten_p:
                ind_ten = ind
                break
        print('Total pat: {}. At 10 percent: {} pat diagnosed, {} pat wrong'.format(aantal_pat, toScreen[1][ind_ten], toScreen[1][ind_ten]- ten_p))
        
    legax = []
    for numax in plots:
        legax.append(numax[0])

    plt.legend(handles = legax)
    plt.xlabel('Patients classified positive (n)')
    plt.ylabel('Early diagnosed patients (n)')
    plt.grid(color='gray', linestyle='-', linewidth=0.3)
    plt.title('1:45 ratio, -24 to -12 months')

### Plot the feature importance

In [ ]:
def plotFeatureImportance(pipe, clf):
    
    # Plot the most important features as given by the model.
    # This part is not yet completly finished
    # Works for the LR (which is needed), still needs work for XGB
    
    import numpy as np
    import matplotlib.pyplot as plt

    full_dict = pipe['sfs'].get_metric_dict()

    paramInOrder = []
    for i in full_dict.keys():
        for j in full_dict[i]['feature_idx']:
            if j not in paramInOrder:
                paramInOrder.append(j)

    allparam = pipe['preprocessing'].transformers[1][2]      
    FFSorder = [allparam[x] for x in paramInOrder]
    included_par = FFSorder[0:len(allparam)]

    importance = clf['classifier'].coef_
    #importance = clf['classifier'].feature_importances_
    sortind = np.argsort(abs(importance[0]))[-20::]
    #sortind = np.argsort(abs(importance))[-20::]
    yax = importance[0,sortind]
    #yax = importance[sortind]
    xax = [included_par[x] for x in sortind]
    plt.barh(xax,yax)
    plt.xlabel('Feature importance (weight)')
    plt.ylabel('Feature')
    
    #sortind = np.argsort(abs(importance[0]))[-20::]
    #yax = importance[0,sortind]
    #xax = [included_par[x] for x in sortind]
    #plt.barh(xax,yax)

# Run the algorithms

In [ ]:
# This is the section which you run to start the plotting
plotInfo={}

# Ratio can be 1to1 or upsampled
# SortCurve can be AUC or PR
# Curve can be on test set or on training set
ratio = 'upsampled'
sortCurve = 'AUC'
curve = 'test'

for setNum in [1,2,3]:
    for modelName in ['LR','RF','XGB']:
        X_train, y_train, X_test, y_test, best_params, pipe, X_train_extra, X_test_extra, y_train_extra, y_test_extra = loadmodules(setNum, modelName)
        Model_train,Model_test, clf = trainModel(X_train, y_train, X_test, y_test, X_train_extra, X_test_extra, y_train_extra, y_test_extra, best_params, modelName, ratio=ratio, sortCurve = sortCurve)
        plotInfo[str(setNum) + modelName] = [Model_train, Model_test]
        
        # Manually alter predictions (which is needed for the pvalue of the AUC) vs the current one. 
        # i.e. Run the best algorithm, save as predictions, and then comment that section and run a different model and print the pvalue
        
        #predictions = Model_test[4] # best
        #print(calc_pvalue_AUCs(Model_test[5], predictions, Model_test[4])) # Current

# Select what you want to do: Plot the ROC, plot the patients screened vs identified, plot feature importance, and save if needed

plotCurve(plotInfo, curve = curve, sortCurve = sortCurve)
#plotNumtoscreenCurve(plotInfo, curve = curve)
#plotFeatureImportance(pipe, clf)
#plt.savefig('XGB_1to30_2.png', dpi=600, bbox_inches='tight')

In [ ]:
# This section can be used for plotting histograms of false / true negatives / positives
'''Under construction
totalPat = 1709
plotname = '3LR'

y_pred = plotInfo[plotname][1][4]
y_true = plotInfo[plotname][1][5]
X_test = pd.concat([X_test, X_test_extra])

import math
patToPredict = math.floor(totalPat/10)

for ind,i in enumerate(plotInfo[plotname][1][1]):
    if i >= (patToPredict/(patToPredict+(totalPat-patToPredict))):
        thresind = ind
        break

thres = plotInfo[plotname][1][2][thresind]
truepos = []
falsepos = []
trueneg = []
falseneg = []
overig=[]

for ind, i in enumerate(y_pred):
    if i>=thres and y_true.iloc[ind]==1:
        truepos.append(ind)
    elif i>=thres and y_true.iloc[ind]==0:
        falsepos.append(ind)
    elif i<thres and y_true.iloc[ind]==0:
        trueneg.append(ind)
    elif i<thres and y_true.iloc[ind]==1:s
        falseneg.append(ind)
    else:
        overig.append(ind)
        
fig, ax = plt.subplots(2,2, sharex = 'col')
ax[0,0].hist(X_test.iloc[truepos,0], bins = range(60,115,2))
ax[0,1].hist(X_test.iloc[falsepos,0], bins = range(60,115,2))
ax[1,0].hist(X_test.iloc[trueneg,0], bins = range(60,115,2))
ax[1,1].hist(X_test.iloc[falseneg,0], bins = range(60,115,2))

ax[0,0].set_ylabel('Number of patients (n)')
ax[1,0].set_ylabel('Number of patients (n)')

ax[1,0].set_xlabel('Age in bins of 2 years')
ax[1,1].set_xlabel('Age in bins of 2 years')

ax[0,0].set_title('True positive patients')
ax[0,1].set_title('False positive patients')
ax[1,0].set_title('True negative patients')
ax[1,1].set_title('False negative patients')

#plt.savefig('Hist_70_chronic_1year_bin2_DS3_LR_1to5.png', dpi=600, bbox_inches='tight')
'''